<a href="https://colab.research.google.com/github/H88X/Pill_Search_App/blob/master/SERVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체
print(device)

cuda:0


In [ ]:
model = models.resnet34(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

PATH = '/content/drive/MyDrive/pill_train/pill_capstone.pth'
model.load_state_dict(torch.load(PATH))

model.eval()
start_time = time.time()


transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
 
data_dir = '/content/drive/MyDrive/dataset'
train_datasets = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)

class_names = train_datasets.classes
print('클래스:', class_names)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

클래스: ['Inhusin_capsule', 'Mode_Cough_S', 'Tylenol_er', 'cefalexin', 'depakote', 'gaborin', 'pensag', 'tagsen', 'tenten', 'tylenol_cold']


In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 14759, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 14759 (delta 45), reused 74 (delta 40), pack-reused 14677
Receiving objects: 100% (14759/14759), 13.52 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (10189/10189), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 40.4 MB/s 


In [ ]:
#yolov5
pt_file_path = "/content/drive/MyDrive/Yolov5_weight/best.pt"
model_yolo = torch.hub.load('/content/yolov5', 'custom', pt_file_path, source='local')

YOLOv5 🚀 v6.2-235-g75728bb Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


플라스크 서버

In [ ]:
# import io
# from flask_ngrok import run_with_ngrok
# from flask import Flask, jsonify, request
# from PIL import Image


# PATH = '/content/drive/MyDrive/pill_train/pill_train.pth'
# model.load_state_dict(torch.load(PATH))

# model.eval()  #eval 모드
# start_time = time.time()


# def get_prediction(image_bytes):
#     image = Image.open(io.BytesIO(image_bytes)) #이미지 불러오기
    
#     image = transforms_test(image).unsqueeze(0).to(device)#이미지를 텐서형식으로 변환하기

#     with torch.no_grad():

#         outputs = model(image)
#         _, preds = torch.max(outputs, 1)

#     return class_names[preds[0]]

# def save_image(file):
#     file.save('/content/drive/MyDrive/test_image'+ file.filename)

# def save_image_r(file):
#     file.save('/content/drive/MyDrive/result_image'+ file.filename)

# app = Flask(__name__)


# @app.route('/', methods=['POST','GET'])
# def predict():
#     if request.method == 'POST':
#         # 이미지 바이트 데이터 받아오기 
#         file = request.files['file']
#         image_bytes = file.read()

#         #save_image(file)
#         #img = Image.open(file)

#         # 이미지 Yolo 처리
#         #yolo_image = model_yolo(img)
#         #result_yolo = yolo_image.crop() #이미지 처리 과정에서 어떤 형식으로 저정되는 지 재확인 필요
#         #print("타입", type(result_yolo))
#         #save_image_r(result_yolo) 

        
        
#         # 분류 결과 확인 및 클라이언트에게 결과 반환
#         class_name = get_prediction(image_bytes=image_bytes)
#         print("결과:", {'class_name': class_name})
#         return jsonify({'class_name': class_name})
        

플라스크 YOLOv5 추가


In [ ]:
import io
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
from PIL import Image
import torchvision.transforms as transforms



PATH = '/content/drive/MyDrive/pill_train/pill_capstone.pth'
model.load_state_dict(torch.load(PATH))

model.eval()  #eval 모드
start_time = time.time()



def get_prediction(image_bytes):
    image = transforms_test(image_bytes).unsqueeze(0).to(device)#이미지를 텐서형식으로 변환하기
    with torch.no_grad():

        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return class_names[preds[0]]

app = Flask(__name__)


@app.route('/', methods=['POST','GET'])
def predict():
    if request.method == 'POST':
        # 이미지 바이트 데이터 받아오기 
        file = request.files['file']
        image_bytes = file.read()
        img = Image.open(file)
        
        # 이미지 Yolo 처리
        yolo_image = model_yolo(img)
        results = yolo_image.crop(save=True, save_dir='/content/drive/MyDrive/capston_source/Yolov5_result', exist_ok=True) #이미지 처리 과정에서 어떤 형식으로 저정되는 지 재확인 필요


        file_path = '/content/drive/MyDrive/capston_source/Yolov5_result/image0.jpg'
        yolo_results = '/content/drive/MyDrive/capston_source/Yolov5_result/crops/Pill/image0.jpg'

        yolo_result_img = Image.open(yolo_results)

        if os.path.exists(file_path):
          os.remove(file_path)
        

        #분류 결과 확인 및 클라이언트에게 결과 반환
        class_name = get_prediction(image_bytes=yolo_result_img)
        print("결과:", {'class_name': class_name})

        if os.path.exists(yolo_results):
          os.remove(yolo_results)

        return jsonify({'class_name': class_name})        

In [ ]:
run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9711-34-143-175-85.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:590: UserWarning: Metadata Warning, tag 282 had too many entries: 2, expected 1
  % (tag, len(values))
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:590: UserWarning: Metadata Warning, tag 283 had too many entries: 2, expected 1
  % (tag, len(values))
Saved 1 image to /content/drive/MyDrive/capston_source/Yolov5_result
Saved results to /content/drive/MyDrive/capston_source/Yolov5_result

127.0.0.1 - - [10/Nov/2022 13:59:49] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'gaborin'}


...

결과: {'class_name': 'tylenol_cold'}


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:590: UserWarning: Metadata Warning, tag 282 had too many entries: 2, expected 1
  % (tag, len(values))
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:590: UserWarning: Metadata Warning, tag 283 had too many entries: 2, expected 1
  % (tag, len(values))
Saved 1 image to /content/drive/MyDrive/capston_source/Yolov5_result
Saved results to /content/drive/MyDrive/capston_source/Yolov5_result

127.0.0.1 - - [10/Nov/2022 14:01:16] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'gaborin'}


Saved 1 image to /content/drive/MyDrive/capston_source/Yolov5_result
Saved results to /content/drive/MyDrive/capston_source/Yolov5_result

127.0.0.1 - - [10/Nov/2022 14:04:57] "POST / HTTP/1.1" 200 -


결과: {'class_name': 'tylenol_cold'}
